# Tutorial 01. Block Poisson problem.

In this tutorial we first solve the problem

\begin{cases}
-u'' = f & \text{in }\Omega = (0, 1),\\
 u   = 0 & \text{on }\Gamma = \{0, 1\}
\end{cases}

using standard FEniCSx code.

Then we use both block support of FEniCSx and multiphenics to solve the system

\begin{cases}
- w_1'' - 2 w_2'' = 3 f & \text{in }\Omega,\\
- 3 w_1'' - 4 w_2'' = 7 f & \text{in }\Omega
\end{cases}

subject to

\begin{cases}
 w_1 = 0 & \text{on }\Gamma,\\
 w_2 = 0 & \text{on }\Gamma
\end{cases}

By construction the solution of the system is
$$(w_1, w_2) = (u, u)$$

We then compare the obtained solutions.

In [ ]:
from numpy import finfo, isclose, where
from petsc4py import PETSc
from ufl import *
from dolfinx import *
from dolfinx.cpp.la import get_local_vectors
from dolfinx.fem import assemble_matrix_block, assemble_scalar, assemble_vector_block, create_vector_block, locate_dofs_topological
from dolfinx.plotting import plot
from multiphenics import *

### Mesh generation

In [ ]:
mesh = UnitIntervalMesh(MPI.comm_world, 32)

In [ ]:
def left(x):
    return abs(x[0] - 0.) < finfo(float).eps

def right(x):
    return abs(x[0] - 1.) < finfo(float).eps

boundaries = MeshFunction("size_t", mesh, mesh.topology.dim - 1, 0)
boundaries.mark(left, 1)
boundaries.mark(right, 1)
boundaries_1 = where(boundaries.values == 1)[0]

In [ ]:
x0 = SpatialCoordinate(mesh)[0]

### Solver parameters

In [ ]:
solver_parameters = {"ksp_type": "preonly", "pc_type": "lu", "pc_factor_mat_solver_type": "mumps"}

### Standard case (solve for $u$ with FEniCSx)

In [ ]:
def run_standard():
    # Define a function space
    V = FunctionSpace(mesh, ("Lagrange", 2))
    u = TrialFunction(V)
    v = TestFunction(V)

    # Define problems forms
    a = inner(grad(u), grad(v))*dx + u*v*dx
    f = 100*sin(20*x0)*v*dx

    # Define boundary conditions
    zero = Function(V)
    with zero.vector.localForm() as zero_local:
        zero_local.set(0.0)
    bdofs_V_1 = locate_dofs_topological(V, mesh.topology.dim - 1, boundaries_1)
    bc = DirichletBC(zero, bdofs_V_1)

    # Solve the linear system
    u = Function(V)
    solve(a == f, u, bc, petsc_options=solver_parameters)
    u.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

    # Return the solution
    return u

In [ ]:
u = run_standard()

In [ ]:
plot(u)

### Standard block case (solve for ($w_1$, $w_2$) with FEniCSx)

In [ ]:
def run_standard_block():
    # Define a block function space
    V1 = FunctionSpace(mesh, ("Lagrange", 2))
    V2 = FunctionSpace(mesh, ("Lagrange", 2))
    (u1, u2) = (TrialFunction(V1), TrialFunction(V2))
    (v1, v2) = (TestFunction(V1), TestFunction(V2))

    # Define problem block forms
    aa = [[1*inner(grad(u1), grad(v1))*dx + 1*u1*v1*dx, 2*inner(grad(u2), grad(v1))*dx + 2*u2*v1*dx],
          [3*inner(grad(u1), grad(v2))*dx + 3*u1*v2*dx, 4*inner(grad(u2), grad(v2))*dx + 4*u2*v2*dx]]
    ff = [300*sin(20*x0)*v1*dx,
          700*sin(20*x0)*v2*dx]

    # Define block boundary conditions
    zero = Function(V1)
    with zero.vector.localForm() as zero_local:
        zero_local.set(0.0)
    bdofs_V1_1 = locate_dofs_topological((V1, V1), mesh.topology.dim - 1, boundaries_1)
    bdofs_V2_1 = locate_dofs_topological((V2, V1), mesh.topology.dim - 1, boundaries_1)
    bc1 = DirichletBC(zero, bdofs_V1_1, V1)
    bc2 = DirichletBC(zero, bdofs_V2_1, V2)
    bcs = [bc1, bc2]

    # Assemble the block linear system
    AA = assemble_matrix_block(aa, bcs)
    AA.assemble()
    FF = assemble_vector_block(ff, aa, bcs)

    # Solve the block linear system
    uu = create_vector_block(ff)
    ksp = PETSc.KSP()
    ksp.create(mesh.mpi_comm())
    ksp.setOperators(AA)
    ksp.setType("preonly")
    ksp.getPC().setType("lu")
    ksp.getPC().setFactorSolverType("mumps")
    ksp.setFromOptions()
    ksp.solve(FF, uu)
    (u1, u2) = (Function(V1), Function(V2))
    with u1.vector.localForm() as u1_local, u2.vector.localForm() as u2_local:
        (u1_local[:], u2_local[:]) = get_local_vectors(
            uu,
            [u1.function_space.dofmap.index_map, u2.function_space.dofmap.index_map]
        )
    u1.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)
    u2.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

    # Return the block solution
    return u1, u2

In [ ]:
u1, u2 = run_standard_block()

In [ ]:
plot(u1)

In [ ]:
plot(u2)

### Multiphenics block case (solve for ($w_1$, $w_2$) with multiphenics)

In [ ]:
def run_multiphenics():
    # Define a block function space
    V = FunctionSpace(mesh, ("Lagrange", 2))
    VV = BlockFunctionSpace([V, V])
    uu = BlockTrialFunction(VV)
    vv = BlockTestFunction(VV)
    (u1, u2) = block_split(uu)
    (v1, v2) = block_split(vv)

    # Define problem block forms
    aa = [[1*inner(grad(u1), grad(v1))*dx + 1*u1*v1*dx, 2*inner(grad(u2), grad(v1))*dx + 2*u2*v1*dx],
          [3*inner(grad(u1), grad(v2))*dx + 3*u1*v2*dx, 4*inner(grad(u2), grad(v2))*dx + 4*u2*v2*dx]]
    ff = [300*sin(20*x0)*v1*dx,
          700*sin(20*x0)*v2*dx]
    AA = BlockForm2(aa, [VV, VV])
    FF = BlockForm1(ff, [VV])

    # Define block boundary conditions
    zero = Function(V)
    with zero.vector.localForm() as zero_local:
        zero_local.set(0.0)
    bdofs_VV0_1 = locate_dofs_topological((VV.sub(0), V), mesh.topology.dim - 1, boundaries_1)
    bdofs_VV1_1 = locate_dofs_topological((VV.sub(1), V), mesh.topology.dim - 1, boundaries_1)
    bc1 = [DirichletBC(zero, bdofs_VV0_1, VV.sub(0))]
    bc2 = [DirichletBC(zero, bdofs_VV1_1, VV.sub(1))]
    bcs = BlockDirichletBC([bc1, bc2], VV)

    # Solve the block linear system
    uu = BlockFunction(VV)
    block_solve(AA, uu, FF, bcs, petsc_options=solver_parameters)
    uu1, uu2 = uu

    # Return the block solution
    return uu1, uu2

In [ ]:
uu1, uu2 = run_multiphenics()

In [ ]:
plot(uu1)

In [ ]:
plot(uu2)

### Error computation between the different cases

In [ ]:
def compute_errors(u1, u2, uu1, uu2):
    u_1_norm = sqrt(MPI.sum(mesh.mpi_comm(), assemble_scalar(inner(grad(u1), grad(u1))*dx)))
    u_2_norm = sqrt(MPI.sum(mesh.mpi_comm(), assemble_scalar(inner(grad(u2), grad(u2))*dx)))
    err_1_norm = sqrt(MPI.sum(mesh.mpi_comm(), assemble_scalar(inner(grad(u1 - uu1), grad(u1 - uu1))*dx)))
    err_2_norm = sqrt(MPI.sum(mesh.mpi_comm(), assemble_scalar(inner(grad(u2 - uu2), grad(u2 - uu2))*dx)))
    print("  Relative error for first component is equal to", err_1_norm/u_1_norm)
    print("  Relative error for second component is equal to", err_2_norm/u_2_norm)
    assert isclose(err_1_norm/u_1_norm, 0., atol=1.e-10)
    assert isclose(err_2_norm/u_2_norm, 0., atol=1.e-10)

In [ ]:
print("Computing errors between standard and standard block")
compute_errors(u, u, u1, u2)
print("Computing errors between standard and multiphenics")
compute_errors(u, u, uu1, uu2)
print("Computing errors between standard block and multiphenics")
compute_errors(u1, u2, uu1, uu2)